In [ ]:
# 실습: 삼성전자 주가 데이터를 분석하는 어플리케이션을 function(tool)으로 수정하여, 다른 종목의 데이터를 정해진 내용대로(종목, 시작날짜, 종료날짜) 가져와서 답하는 어플리케이션을 작성하시오.
# 예) LG전자(066570)의 2025년 6월 주가는 어땠지?
# (주)에이피알(278470)의 2024년 12월부터 2025년 1월까지 주가를 분석해줘.

In [ ]:
!pip install pykrx

In [ ]:
from pykrx import stock
from pykrx import bond

In [ ]:
import os
import json
from openai import OpenAI

client = OpenAI()
client.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
import re
from datetime import datetime
import pandas as pd
from pykrx import stock
import json
import os
import openai

# OpenAI 키 환경변수로 설정되어 있어야 함
openai.api_key = os.getenv("OPENAI_API_KEY")


def parse_query_dates(query):
    """
    query 문장에서 YYYY년 MM월 형태 날짜 구간 추출.
    1개면 해당 월 전체, 2개면 범위로 간주
    """
    date_ranges = re.findall(r'(\d{4})년\s*(\d{1,2})월', query)

    if len(date_ranges) == 1:
        year, month = date_ranges[0]
        start_date = datetime(int(year), int(month), 1)
        # 다음달 첫날
        if int(month) == 12:
            end_date = datetime(int(year) + 1, 1, 1)
        else:
            end_date = datetime(int(year), int(month) + 1, 1)
    elif len(date_ranges) == 2:
        year1, month1 = date_ranges[0]
        year2, month2 = date_ranges[1]
        start_date = datetime(int(year1), int(month1), 1)
        if int(month2) == 12:
            end_date = datetime(int(year2) + 1, 1, 1)
        else:
            end_date = datetime(int(year2), int(month2) + 1, 1)
    else:
        raise ValueError("날짜 정보를 정확히 입력해주세요. ex) 2025년 6월 또는 2024년 12월부터 2025년 1월까지")

    # end_date에서 하루 빼서 해당월 마지막 날로 맞춤
    end_date = end_date - pd.Timedelta(days=1)

    return start_date.strftime("%Y%m%d"), end_date.strftime("%Y%m%d")


def parse_query_code_name(query):
    """
    종목명과 6자리 종목코드 추출
    """
    m = re.search(r'([\w\.\(\)]+)\((\d{6})\)', query)
    if not m:
        raise ValueError("종목명과 6자리 코드를 찾을 수 없습니다.")
    name, code = m.group(1), m.group(2)
    return name, code


def get_stock_data_krx(code, start, end):
    """
    pykrx 로 주가 데이터 가져오기
    start, end는 "YYYYMMDD" 문자열
    """
    df = stock.get_market_ohlcv_by_date(start, end, code)
    if df.empty:
        raise ValueError("해당 기간에 주가 데이터가 없습니다.")
    df.reset_index(inplace=True)
    df['날짜'] = df['날짜'].dt.strftime('%Y-%m-%d')
    return df


def save_csv_and_json(df, code, start, end):
    """
    CSV 저장 및 JSON 변환 (OpenAI 입력용)
    """
    filename = f"{code}_{start}_{end}.csv"
    df.to_csv(filename, index=False, encoding='utf-8-sig')

    # 필요한 컬럼만 json 변환
    json_data = df[['날짜', '시가', '고가', '저가', '종가', '거래량']].to_dict(orient='records')

    return filename, json_data


def ask_gpt(query, json_data):
    system_prompt = "너는 주식 전문가야. 아래 주가 데이터를 분석해줘."
    user_prompt = f"질문: {query}\n\n데이터: {json.dumps(json_data, ensure_ascii=False)}"

    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0,
        max_tokens=1024,
    )
    return response.choices[0].message.content


if __name__ == "__main__":
    # 사용자 문의 1
    query = "(주)에이피알(278470)의 2024년 12월부터 2025년 1월까지 주가를 분석해줘"
    # 나중에 query = "(주)에이피알(278470)의 2024년 12월부터 2025년 1월까지 주가를 분석해줘. LG전자(066570)의 2025년 6월 주가는 어땠지?" 로 바꾸면 됨

    try:
        name, code = parse_query_code_name(query)
        start, end = parse_query_dates(query)
        df = get_stock_data_krx(code, start, end)
        csv_file, json_data = save_csv_and_json(df, code, start, end)

        print(f"CSV 파일 저장됨: {csv_file}")
        print(f"주가 데이터 (JSON 일부): {json.dumps(json_data[:3], ensure_ascii=False, indent=2)}")  # 일부 출력

        # GPT 호출
        answer = ask_gpt(query, json_data)
        print("\nGPT 응답:")
        print(answer)

    except Exception as e:
        print(f"오류 발생: {e}")

CSV 파일 저장됨: 278470_20241201_20250131.csv
주가 데이터 (JSON 일부): [
  {
    "날짜": "2024-12-02",
    "시가": 49250,
    "고가": 51000,
    "저가": 48300,
    "종가": 48950,
    "거래량": 287940
  },
  {
    "날짜": "2024-12-03",
    "시가": 48950,
    "고가": 53400,
    "저가": 48900,
    "종가": 52600,
    "거래량": 830198
  },
  {
    "날짜": "2024-12-04",
    "시가": 51000,
    "고가": 52900,
    "저가": 49850,
    "종가": 51100,
    "거래량": 549582
  }
]

GPT 응답:
(주)에이피알(278470)의 2024년 12월부터 2025년 1월까지의 주가 데이터를 분석해보겠습니다.

### 1. 주가 동향
- **2024년 12월 초**: 12월 2일에 종가가 48,950원이었고, 이후 12월 3일에는 52,600원으로 급등했습니다. 이 시점에서 주가는 상승세를 보였습니다.
- **12월 중순**: 12월 11일에는 54,900원으로 최고점을 기록한 후, 12월 12일에도 55,300원으로 유지되었습니다. 이 시점에서 거래량도 증가하여 투자자들의 관심이 높아졌음을 알 수 있습니다.
- **12월 하순**: 12월 20일에는 47,250원으로 급락하며 하락세로 전환되었습니다. 이후 12월 30일에는 50,000원으로 반등했지만, 여전히 불안정한 모습을 보였습니다.

### 2. 2025년 1월 동향
- **1월 초**: 1월 2일에는 51,100원으로 시작하여, 1월 3일에는 54,100원으로 다시 상승세를 보였습니다. 그러나 이후 1월 10일에는 52,100원으로 하락했습니다.
- **1월 중순**: 1월 13일에는 52,300원으로 소폭 상승했으나, 1월 15일에는 50,200원으

In [ ]:
def get_current_weather(location, unit="섭씨"):
    weather_info = {
        "location": location,
        "temperature": "24",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

messages = [{"role": "user", "content": "지금 서울날씨를 섭씨로 알려줘."}]
functions = [
    {
        "name": "get_current_weather",
        "description": "특정 지역의 날씨를 알려줍니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "지역이름. 예) 서울, 부산, 제주도",
                },
                "unit": {"type": "string", "enum": ["섭씨", "화씨"]},
            },
            "required": ["location"],
        },
    }
]

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    functions=functions,
    function_call="auto",
    )
response_message = response.choices[0].message
print(response_message)

In [ ]:
if response_message.function_call is not None:
    # Note: the JSON response may not always be valid; be sure to handle errors
    available_functions = {
        "get_current_weather": get_current_weather,
    }
    function_name = response_message.function_call.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(
        location=function_args['location'],
        unit=function_args['unit']
    )

    messages.append(response_message)
    messages.append(
        {
            "role": "function",
            "name": function_name,
            "content": function_response,
        }
    )
    second_response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    )  # get a new response from GPT where it can see the function response

    print(second_response.choices[0].message.content)